# PhiSpy

This is a Jupyter Notebook that shows how to run PhiSpy manually. You can run through all the steps that PhiSpy takes to determine whether a genome contains a prophage, and inspect all of the data generated by PhiSpy.

You will need to install [PhiSpy](https://github.com/linsalrob/PhiSpy#installation), [Jupyter Notebooks](https://jupyter.org/install), and [pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/install.html)

<small>Note: PhiSpy does not normally use pandas, but we use it here to visualize the data!</small>

In [1]:
# set up the environment
import os
import sys
import gzip
from functools import reduce
import tempfile

import pandas as pd

from Bio import SeqIO

import PhiSpyModules

### Check the PhiSpy version

We recommend at least version 4.0.3 but preferable 4.1 or higher

In [2]:
print("PhiSpy version: " + PhiSpyModules.__version__)

PhiSpy version: 4.1.10


### Define your genbank file here.

You may need to set the full path to the file. You can use gzip compressed file or an uncompressed file. Obviously, you will know whether it is compressed or not, but this demonstrates how PhiSpy determines when a file is compressed.

**This should be the only line you need to change to run PhiSpy completely!**

In [3]:
genbankfile = "/home/redwards/GitHubs/PhiSpy/test_genbank_files/Streptococcus_pyogenes_M1_GAS.gb.gz"

### Parse the file

We use BioPython to parse the file, but we also add a few additional mehtods to the standard BioPython object to ease parsing. We also merge or split compound features (those with more than one location along the chromosome) to appropriately handle them.

Our `record` object is an extended `SeqIO.parse` object.

In [4]:
min_contig_size = 1000

if PhiSpyModules.is_gzip_file(genbankfile):
    handle = gzip.open(genbankfile, 'rt')
else:
    handle = open(genbankfile, 'r')

record  = PhiSpyModules.SeqioFilter(filter(lambda x: len(x.seq) > min_contig_size, SeqIO.parse(handle, "genbank")))
handle.close()

# we check to make sure there are some contigs left to process
ncontigs = reduce(lambda sum, element: sum + 1, record, 0)
print(f"There are {ncontigs} contigs to predict prophages on!")

There are 1 contigs to predict prophages on!


## Define the parameters that we will use

These are normally provided as command line options, but for jupyter we set them here

paramter | meaning | options | default value
--- | --- | --- | --
kmers_type | What do we count kmers with? | `all`, `codon`, `simple` | `all`
window_size | How many consecutive ORFs to include? | an integer | 30
record | the `Bio.SeqIO` object with all the sequences | a `Bio.SeqIO` object | `record`
expand_slope | whether to use the square of the slope of the Shannon scores | `True` or `False` | `False`
number | Number of consecutive genes in a region of window size that must be prophage genes | an integer | 5
nonprophage_genegaps | The number of non phage genes betweeen prophages | an integer | 10
quiet | Don't make additional outputs | `True` or `False` | `True`


*Note*: You can add an additional paramter, `make_training_data` here (its actual value doesn't matter) that will append an additional column to the output for each ORF that includes a `1` (`True`) if the ORF is thought or stated to be a phage gene or `0` (`False`) otherwise.

In [5]:
parameters = {
    'kmers_type': "all",
    'window_size': 30,
    'record': record,
    'expand_slope': False,
    'training_set': "data/trainSet_genericAll.txt",
    'randomforest_trees': 500,
    'threads': 4,
    'quiet': True,
    'nonprophage_genegaps': 10,
    'number': 5,
    'color' : True,
    'debugpp' : 'all_pp_jup.tsv',
    'evaluate':False,
    'make_training_data':None,
    'skip_search':False,
    'phmms':None,
    'phage_genes' : 2,
    'metrics' : ['orf_length_med', 'shannon_slope', 'at_skew', 'gc_skew', 'max_direction'],
}

## Generate the test data

This is the step that actually does all the measurements!

In this example, we convert the output to a pandas dataframe for visualization and exploration.

In [6]:
parameters['test_data'] = PhiSpyModules.measure_features(**parameters)
test_data = parameters['test_data']
# note that if you include make_training_data you will need to add an "is_phage" column here
test_df = pd.DataFrame(parameters['test_data'], columns = ['orf_length_med', 'shannon_slope', 'at_skew',
                                             'gc_skew', 'max_direction', 'phmms'])
test_df.head()

,orf_length_med,shannon_slope,at_skew,gc_skew,max_direction,phmms
0,393.0,0.0,2.305653,1.598561,14,0.0
1,382.5,0.0,2.379462,1.520969,15,0.0
2,372.0,0.0,2.108348,1.521077,16,0.0
3,295.5,0.0,1.913390,1.503681,17,0.0
4,372.0,0.0,1.886275,1.561022,18,0.0


### Run the random forest

Here we run the random forest to identify the phages, and combine that into our initial table as the `rank` column.

In [7]:
parameters['rfdata'] = PhiSpyModules.call_randomforest(**parameters)
parameters['initial_tbl'] = PhiSpyModules.make_initial_tbl(**parameters)
parameters['output_dir'] = tempfile.mkdtemp()
initial_table_df = pd.DataFrame(parameters['initial_tbl'], columns = ['gene id', 'function', 'contig', 'start', 'stop', 'position', 'rank', 'my status', 'pp'])
initial_table_df.head()

,gene id,function,contig,start,stop,position,rank,my status,pp
0,[231:1587](+),Chromosomal replication initiator protein DnaA,NC_002737,232,1587,0,0.0000,0,0.0
1,[1741:2878](+),DNA polymerase III beta subunit (EC 2.7.7.7),NC_002737,1742,2878,1,0.0000,0,0.0
2,[2952:3150](+),Uncharacterized protein CAC3725,NC_002737,2953,3150,2,0.0000,0,0.5
3,[3295:3415](-),hypothetical protein,NC_002737,3415,3296,3,0.0000,0,0.5
4,[3479:4595](+),GTP-binding and nucleic acid-binding protein YchF,NC_002737,3480,4595,4,0.0001,0,0.0


In [8]:
len(parameters['test_data'])

1853

In [12]:
parameters['rfdata'].shape

(1853,)

In [13]:
type(parameters['rfdata'])

numpy.ndarray

In [18]:
np.zeros(len(parameters['test_data'])).shape

(1853,)

In [17]:
len(parameters['test_data'])

1853

In [52]:
from io import TextIOWrapper
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def call_randomforest_local(**kwargs):
    training_file = kwargs['training_set_full']
    test_data = kwargs['test_data']

    if not os.path.exists(training_file):
        sys.stderr.write(f"Training file {training_file} not found. Can't continue\n")
        return None
    
    strm = open(training_file, 'rb')
    train_data = np.genfromtxt(TextIOWrapper(strm), delimiter="\t", skip_header=1, filling_values=1)

    all_metrics = ['orf_length_med', 'shannon_slope', 'at_skew', 'gc_skew', 'max_direction', 'phmms']
    if kwargs['phmms']:
        kwargs['metrics'].append('phmms')

    if len(kwargs['metrics']) < len(all_metrics):
        sys.stderr.write(f"Using the following metric(s): {', '.join(sorted(kwargs['metrics']))}.\n")
        skip_metrics = [all_metrics.index(x) for x in set(all_metrics) - set(kwargs['metrics'])]
        train_data = np.delete(train_data, skip_metrics, 1)
        test_data = np.delete(test_data, skip_metrics, 1)
    else:
        sys.stderr.write(f"Using all metrics: {', '.join(all_metrics)}.\n")

    """
    Przemek's comment
    by default 10 until version 0.22 where default is 100
    number of estimators also implies the precision of probabilities, generally 1/n_estimators
    in R's randomForest it's 500 and the usage note regarding number of trees to grow says:
    "This should not be set to too small a number, to ensure that every input row gets predicted at least a few times."
    """
    sys.stderr.write(f"Train data is {train_data.shape}\n")
    if train_data.shape[1] > 1:
        clf = RandomForestClassifier(n_estimators=kwargs['randomforest_trees'], n_jobs=kwargs['threads'])
        clf.fit(train_data[:, :-1], train_data[:, -1].astype('int'))
        return clf.predict_proba(test_data)[:,1]
    else:
        return np.zeros(len(kwargs['test_data']))


In [ ]:
parameters['test_data'] = test_data
parameters['training_set_full'] = '/home/redwards/GitHubs/PhiSpy/PhiSpyModules/data/trainSet_genericAll.txt'
#parameters['metrics'] = ["xxxx"]
parameters['metrics'] = ['orf_length_med', 'shannon_slope', 'at_skew', 'gc_skew', 'max_direction', 'phmms']

clf = call_randomforest_local(**parameters)
clf.shape

In [35]:
r = clf.predict_proba(test_data)
r.shape

ValueError: Number of features of the model must match the input. Model n_features is 5 and input n_features is 6 

### Refine the predictions

Finally, we refine the predictions from the random forest and other metrics, and then predict the *att* sequences.

In [ ]:
parameters['pp'] = PhiSpyModules.fixing_start_end(**parameters)
pp_df = pd.DataFrame.from_dict(parameters['pp']).transpose()
pp_df

Our `pp_df` data frame has our final prophage predictions for this genome! 

### Make the final table

Here we just append the pp number of the prophage

In [ ]:
parameters['final_tbl'] = []
for i in parameters['initial_tbl']:
    my_fs = PhiSpyModules.evaluation.check_pp(i[2], i[3], i[4], parameters['pp'])
    parameters['final_tbl'].append(i + [my_fs])

    final_df  = pd.DataFrame(parameters['final_tbl'], columns = ['gene id', 'function', 'contig', 'start', 'stop', 'position', 'rank', 'my status', 'pp', 'final status'])
final_df.head()

In [ ]:
phage_regions = final_df.loc[final_df['final status'] > 0]
phage_regions

In [ ]:
rp = initial_table_df.loc[initial_table_df['function'].str.contains(pat = 'ribosomal protein')]
rp